## This notebook is an example of how to pipeline two models. 
This notebook is an example how to use DeGirum PySDK to do AI inference of a graphical file using two AI models: face detection and mask detection. The face detection model is run on the image and the results are then processed by the mask detection model, one face bbox at a time.
Combined result is then displayed.
This script uses PIL as image processing backend.


In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools

### Specify inference option here

In [ ]:
# Please uncomment and edit one of the following inference options to specify your system configuration case according to
# https://cs.degirum.com/doc/0.5.0/degirum.html#system-configuration-for-specific-use-cases

# 1. DeGirum Cloud Zoo inference:
#zoo = dg.connect_model_zoo("dgcps://cs.degirum.com", token=mytools.token_get())

# 2. AIServer inference via IP address using models from DeGirum Cloud model zoo
#zoo = dg.connect_model_zoo(("192.168.0.7", "https://cs.degirum.com/degirum_com/public"), token=mytools.token_get())

# 3. AIServer inference via IP address using local model zoo
#zoo = dg.connect_model_zoo("192.168.0.1")

# 4. ORCA board installed locally using models from DeGirum Cloud Model Zoo
#zoo = dg.connect_model_zoo("https://cs.degirum.com/degirum_com/public", token=mytools.token_get())

# 5. Local inference with locally deployed model
#zoo = dg.connect_model_zoo("full/path/to/model.json")

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")
mask_det_model = zoo.load_model("mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca_1")

# adjust some model properties
face_det_model.output_confidence_threshold = 0.7
face_det_model.overlay_font_scale = 1.5
face_det_model.overlay_alpha = 1
mask_det_model.overlay_show_labels = True
mask_det_model.overlay_font_scale = 1.5
mask_det_model.overlay_alpha = 1
mask_det_model.output_top_k = 1

In [ ]:
# detect persons
faces = face_det_model('./images/Mask1.jpg')

# for each detected face detect mask
with mask_det_model: # performance optimization to keep connection to mask_det_model open
    for face in faces.results:
        face_box = faces.image.crop(face['bbox'])
        mask = mask_det_model(face_box)
        face["label"] = mask.results[0]["label"]
        display(mask.image_overlay)

# display combined results
display(faces.image_overlay)